In [50]:
import json
import re
from bs4 import BeautifulSoup
import urllib
import psycopg2 as pg2



In [51]:
soup = BeautifulSoup(urllib.request.urlopen('http://ohhla.com/all.html').read(), 'html.parser')

In [52]:
soup.select('pre > a')[0:5]

[<a name="xx"> </a>,
 <a href="anonymous/10_kans/">10 K.A.N.'s</a>,
 <a href="anonymous/10sion/">10sion</a>,
 <a href="anonymous/113/">113</a>,
 <a href="anonymous/eleven_5/">11/5</a>]

In [53]:
art_dic = {}
for a in soup.select('pre > a'):
    try:
        line = str(a).split('">')
        url = line[0].split('<a href="')[1]
        art = line[1].split('<')[0]
        art_dic[art] = url
    except:
        pass

In [62]:
estconn = pg2.connect(database='rap_songs', user='keenan', host='localhost', password='keenan')
s = '''drop table if exists all_artist_names'''
f = '''create table all_artist_names (artist_nm text not null, artist_link text not null, unique (artist_nm, artist_link))'''
commands = []
for art, url in art_dic.items():
    add = '''insert into all_artist_names values (%(art)s, %(url)s)''', {'art':art, 'url':url}
    commands.append(add)
    
cur = estconn.cursor()
cur.execute(s)
cur.execute(f)
for command in commands:
    cur.execute(command[0], command[1])
cur.close()
estconn.commit()
estconn.close()